In [ ]:
from contrans import contrans
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import json
import requests
#dotenv.load_dotenv()
congresskey = os.getenv('congresskey')

In [ ]:
ct = contrans()

In [ ]:
goodbioguide = ct.get_bioguide(name='good', state='Virginia', district=5)
goodbioguide

In [ ]:
goodid = goodbioguide.reset_index()['bioguideId'][0]

In [ ]:
goodid

In [ ]:
billurl= goodsbills = ct.get_sponsoredlegislation(goodid)

In [ ]:
goodsbills[0]['url'] ###this is an example f one bill


In [ ]:
ct.get_billdata(billurl)

In [ ]:
def get_billdata(billurl):
    r = requests.get(billurl,
                     params={'api_key': congresskey})
    bill_json = json.loads(r.text)
    texturl = bill_json['bill']['textVersions']['url']
    r = requests.get(texturl,
                 params={'api_key': congresskey})
    toscrape= json.loads(r.text)['textversions'][0]['formats'][0]['url']
    r = requests.get(toscrape)
    mysoup = BeautifulSoup(r.text, 'html.parser')
    billtext = mysoup.text
    bill_json['text'] = billtext
    return bill_json


In [ ]:
r = requests.get(goodsbills[0]['url'],
                 params={'api_key': congresskey})
texturl= json.loads(r.text)['bill']['textVersions']['url']

r = requests.get(texturl,
                 params={'api_key': congresskey})
toscrape= json.loads(r.text)['textversions'][0]['formats'][0]['url']


In [ ]:
from bs4 import BeautifulSoup

r = requests.get(toscrape)
mysoup = BeautifulSoup(r.text, 'html.parser')
billtext = mysoup.text

In [ ]:
print(mysoup.text)

In [ ]:
json.loads(r.text)['sponsoredLegislations'][0]

In [ ]:
bioguides[['name', 'state', 'district', 'bioguideId']]

In [ ]:
bioguides = ct.get_bioguideIDs()
bioguides

In [ ]:
members_df = ct.get_bioguideIDs()

In [ ]:
members_df

In [ ]:
votes = ct.get_votes()
votes

In [ ]:
house = votes.query('chamber == "House"')
house = house[['rollnumber','icpsr','cast_code']]
house

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat

In [ ]:

house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree':'mean'})
house_mat = house_mat.reset_index()
house_mat

In [ ]:
ideo = ct.get_ideology()
members = ideo[['icpsr', 'party_code', 'bioname']]
members